# Elastic Stack
L'objectif du TP est de continuer à prendre en main la stack Elastic(elasticsearch & Kibana) en mode local avec des requêtes d'agrégation

##  1- elasticsearch
Utiliser l'environnement déja mis en place lors du TP1 et utilisé lors du TP2

Vérifier que l'environnement est bien lancé :

* L'url d'elasticsearch:  http://localhost:9200
* L'url cerebro:  http://localhost:9000
* L'url de Kibana : http://localhost:5601

## 2- Aggregations

* 1- Ecrire l'aggrégation qui permet de chercher dans l'index, le nombre de films par année.

In [ ]:
POST movies/_search
{
    "aggs" : {
      "group_by_year": {
        "terms": {
          "field": "fields.year"
        }
      }
    },
    "size": 0
}

* 2- Donner la note (rating) moyenne des films.

In [ ]:
POST movies/_search
{
    "aggs": {
        "avg_ratings": {
            "avg": {
                "field": "fields.rating"
            }
        }
    },
    "size": 0
}

* 3- Donner la note (rating) moyenne, et le rang moyen des films de George Lucas.

In [ ]:
POST movies/_search
{
    "query": {
        "terms": {
            "fields.directors.keyword": ["George Lucas"]
        }
    },
    "aggs": {
        "directors": {
            "terms": {
                "field": "fields.directors.keyword"
            },
            "aggs": {
                "avg_rating": { "avg": { "field": "fields.rating" } },
                "avg_rank": { "avg": { "field": "fields.rank" } }
            }
        }
    },
    "size": 0
}

* 4- Donnez la note (rating) moyenne des films par année. Attention, il y a ici une imbrication d’agrégats.

In [ ]:
#POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { "field": "fields.year" },
        "aggs": {
          "avg_rating": { "avg": { "field": "fields.rating" } } 
		}
      }
    },
    "size": 0
}


 * 5- Donner la note (rating) minimum, maximum et moyenne des films par année.

In [ ]:
POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { "field": "fields.year" },
        "aggs": {
          "avg_rating": { "avg": { "field": "fields.rating" } } ,
          "min_rating": { "min": { "field": "fields.rating" } } ,
          "max_rating": { "max": { "field": "fields.rating" } } 
		}
      }
    },
    "size": 0
}


* 6- Donner le rang (rank) moyen des films par année et trier par ordre décroissant.

In [ ]:
POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { 
          "field": "fields.year",
          "order" : { "avg_rank" : "desc" } 
        },
        "aggs": {
          "avg_rank": { "avg": { "field": "fields.rank" } } 
		}
      }
    },
    "size": 0
}


* 7- Compter le nombre de films par tranche de note (0-1.9, 2-3.9, 4-5.9...). 

Indication : group_range.


In [ ]:
# version élborée
POST movies/_search
{
  "size": 0,
  "aggs": {
    "rating_ranges": {
      "range": {
        "field": "fields.rating", 
        "script": {
          "lang": "painless",
          "source": "Math.round(_value*100)/100.0"
        },
        "ranges": [
          { "from": 0.0, "to": 1.9 },
          { "from": 2, "to": 3.9 },
          { "from": 4, "to": 4.9 },
          { "from": 5, "to": 6.9 },
          { "from": 7, "to": 8.9 },
          { "from": 9 }
        ]
      }
    }
  }
}

## version approx avec histo
POST movies/_search
{
    "aggs" : {
        "ratings_per_interval": {
            "histogram": {
                "field": "fields.rating",
                "interval": 1.9
            }
        }
    },
    "size": 0
}

 * 8- grouper par “genre” de film, et donner leurs occurrences :
 
indication : un mapping des propriétes peut être nécessaire..
 

In [ ]:
POST movies/_search
{
    "aggs" : {
      "group_by_genre": {
        "terms": {
          "field": "fields.genres.keyword"
        }
      }
    },
    "size": 0
}

### Bonus

   * Donner le nombre d’occurrences de chaque réalisateur ou réalisatrice.

In [ ]:
POST movies/_search
{
    "aggs" : {
      "group_by_genre": {
        "terms": {
          "field": "fields.directors.keyword"
        }
      }
    },
    "size": 0
}

   * Donner la note (rating) moyenne, le rang min et max, des films par acteur. 

In [ ]:

POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { "field": "fields.actors.keyword" },
        "aggs": {
          "avg_rating": { "avg": { "field": "fields.rating" } } ,
          "min_rank": { "min": { "field": "fields.rank" } } ,
          "max_rank": { "max": { "field": "fields.rank" } } 
		}
      }
    },
    "size": 0
}


   * Nombre de réalisateurs distincts pour les films d’aventure.

In [ ]:
POST movies/_search
{
	"query":{ "match_phrase":{"fields.genres":"adventure"} },
	"aggs" : {
		"actors_uniq_count" : {
		  "value_count": { "field" : "fields.actors.keyword" }
		}
	},
	  "size": 0
}

   * Termes les plus utilisés (agrégat : significant_terms) dans les descriptions des films de George Lucas.

* Donner la note moyenne par genre,

In [ ]:
POST movies/_search
{
    "aggs": {
      "avg_rating_per_year": {
        "terms": { "field": "fields.genres.keyword" ,"size":100},
        "aggs": {
          "avg_rating": { "avg": { "field": "fields.rating" } }
		}
      }
    },
    "size": 0
}

* Donner une note minimale, maximale et moyenne pour chaque genre,

In [ ]:
POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { "field": "fields.genres.keyword" ,"size":1000},
        "aggs": {
          "avg_rating": { "avg": { "field": "fields.rating" } } ,
          "min_rating": { "min": { "field": "fields.rating" } } ,
          "max_rating": { "max": { "field": "fields.rating" } } 
		}
      }
    },
    "size": 0
}

* Donner le classement moyen des films par an et les trier par ordre ascendant,

In [ ]:
POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { 
          "field": "fields.year",
          "size":1000,
          "order" : { "avg_rank" : "asc" } 
        },
        "aggs": {
          "avg_rank": { "avg": { "field": "fields.rank" } } 
		}
      }
    },
    "size": 0
}

* Donner le rang moyen du film et la note moyenne du film pour chaque réalisateur. Trier le résultat de manière décroissante en moyenne de rating

In [ ]:
POST movies/_search
{
    "aggs": {
      "rating_films_per_year": {
        "terms": { 
          "field": "fields.directors.keyword",
          "size":1000,
          "order" : { "avg_ranting" : "desc" } 
        },
        "aggs": {
          "avg_rank": { "avg": { "field": "fields.rank" } } ,
          "avg_ranting": { "avg": { "field": "fields.rating" } } 
		}
      }
    },
    "size": 0
}

* Donner les termes occurrences extraites du titre de chaque film. 

# 3- Visualiser les résultats avec Kibana

lancer kibana : https://www.elastic.co/fr/downloads/kibana

Vérifier que c'est bien démarré via l'URL:  http://localhost:5601

RQ: Analyser les diff paramètres dans le fichier de conf: config/kibana.yml

Charger votre index movies dans Elasticsearch et contruire un Dashboard avec 3 Graphes pertinents de votre choix

# Ressources utiles:

voir https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations.html
https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-put-mapping.html

